In [1]:
import pandas as pd
from unidecode import unidecode

ModuleNotFoundError: No module named 'unidecode'

In [2]:
con = pd.read_csv('../dados/Consulta2025_1.csv', encoding='latin1', sep='\t')
con = con.drop(columns=['Horários de preferência'])


oferta = pd.read_csv('../dados/oferta_2025_1_depto-estatística---brasília_graduação.csv')
oferta['disciplina'] = oferta['disciplina'].apply(unidecode)
professores_removidos = ['NICOLLAS STEFAN SOARES DA COSTA', 'ERITON BARROS DOS SANTOS', 'LEANDRO TAVARES CORREIA']
oferta = oferta[~oferta['docente'].isin(professores_removidos)]

dicionario = pd.read_csv('../dados/dicionario.csv')[['disciplina', 'codigo_disciplina']]

In [3]:
oferta['disciplina_turma'] = oferta['codigo_disciplina'] + ' - ' + oferta['codigo_turma'].astype(str)
oferta['leciona'] = 1

In [4]:
colunas = pd.DataFrame({'disciplina':con.columns})
colunas['disciplina'] = colunas['disciplina'].apply(unidecode).str.upper()
colunas.drop_duplicates(subset='disciplina', inplace=True)
colunas = colunas.merge(dicionario, on='disciplina', how='left').fillna('ID')

In [ ]:
con.columns = colunas['codigo_disciplina'].values
mapa = {1:3, 3:1, 'Em princípio, não tenho interesse.':0}
con = con.replace(mapa)

In [6]:
matriz_resultado = oferta.pivot(index='docente',columns='disciplina_turma', values='leciona').fillna(0).astype(int)

In [12]:
oferta['docente_disciplina_turma'] = oferta['docente'] + ' - ' + oferta['disciplina_turma']

In [56]:
matriz_resultado = oferta.pivot(index='docente_disciplina_turma',columns='disciplina_turma', values='leciona').fillna(0).astype(int)
matriz_resultado.index = matriz_resultado.index.str.split('-').str[0].str.strip()

In [57]:
(matriz_resultado.sum(axis=1, numeric_only=True) > 1).any()

False

In [59]:
con.to_csv('../dados/consulta_transformada.csv', index=False, encoding='utf-8-sig')
matriz_resultado.to_csv('../dados/oferta_transformada.csv', index=True, encoding='utf-8-sig')